In [1]:
import os
#os.chdir("/home/tlehella/rotor_checkmate/")
os.chdir("/home/theotime/Documents/Projets/rotor_checkmate/")
import torch
import rkgb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device : {device}")

import torch

device : cpu


# UNET

In [16]:
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)
inp = torch.randn((1,3,256,256))

Using cache found in /home/theotime/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [28]:
rkgb_res = rkgb.make_all_graphs(model,inp,check_device_is_gpu=False)
sg = rkgb_res.S_graph
kg : rkgb.Ktools.K_graph = rkgb_res.K_graph

In [31]:
dict_info = sg.dict_info
for sn in sg.nodes:
    mt = sn.main_target
    mem_inputs = rkgb.utils.imports_from_rotor.MemSize(0)
    for req_sn in sn.deps:
        try:
            mem_inputs += dict_info[req_sn.main_target].memsize
        except: pass
    has_phantoms = f"{mt} phantoms" in kg.dict_kn
    phantom_mem = kg.dict_kn[f"{mt} phantoms"].mem if has_phantoms else 0
    print(f"For {sn.main_target}, "\
          f"inputs : {mem_inputs}, "\
          f"output : {dict_info[sn.main_target].memsize}, "\
          f"phantoms : {phantom_mem}")

For __28_input, inputs : 0.0B, output : 8.0MiBphantoms : 0
For __40_input, inputs : 8.0MiB, output : 8.0MiBphantoms : 0
For __44_input, inputs : 8.0MiB, output : 8.0MiBphantoms : 0
For __57_input, inputs : 8.0MiB, output : 8.0MiBphantoms : 0
For __60_input, inputs : 8.0MiB, output : 2.0MiBphantoms : 0.0B
For __73_input, inputs : 2.0MiB, output : 4.0MiBphantoms : 0
For __85_input, inputs : 4.0MiB, output : 4.0MiBphantoms : 0
For __89_input, inputs : 4.0MiB, output : 4.0MiBphantoms : 0
For __102_input, inputs : 4.0MiB, output : 4.0MiBphantoms : 0
For __105_input, inputs : 4.0MiB, output : 1.0MiBphantoms : 0.0B
For __118_input, inputs : 1.0MiB, output : 2.0MiBphantoms : 0
For __130_input, inputs : 2.0MiB, output : 2.0MiBphantoms : 0
For __134_input, inputs : 2.0MiB, output : 2.0MiBphantoms : 0
For __147_input, inputs : 2.0MiB, output : 2.0MiBphantoms : 0
For __150_input, inputs : 2.0MiB, output : 512.0KiBphantoms : 0.0B
For __163_input, inputs : 512.0KiB, output : 1.0MiBphantoms : 0
For _

In [4]:
pg = rkgb.Ptools.S_to_P_init(rkgb_res.S_graph)
rkgb.Ptools.rule_group_sequences(pg)
rkgb.Ptools.rule_merge_small_flows(pg)

In [5]:
rkgb.Ptools.print_P_graph(pg,name="P_graph UNet")

Partitioned forward graph : 9 nodes


kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found


In [7]:
rkgb.print_graph(rkgb_res.S_graph)

Simplified forward graph : 50 nodes


kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found


# GPT

In [8]:
from models.GPT import GPT2

mod_gpt2 = GPT2(nlayers=12,dropout=0.1)
mod_gpt2.to(device)
input_gpt2 = torch.randint(5400,(100,20),device = device)

In [9]:
import warnings ; warnings.filterwarnings("ignore")
rkgb_on_gpt2 = rkgb.make_all_graphs(mod_gpt2,input_gpt2,check_device_is_gpu=False)
sg = rkgb_on_gpt2.S_graph

In [14]:
pg = rkgb.Ptools.S_to_P_init(sg)
rkgb.Ptools.rule_group_sequences(pg)
rkgb.Ptools.rule_merge_small_flows(pg)

In [15]:
rkgb.print_graph(pg,name="P_graph GPT2_12")

Partitioned forward graph : 20 nodes


kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found


In [33]:
print(sum(pn.size() for pn in pg.list_nodes))

105


kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found


# FNO

In [7]:
from def_FNO import FNO1d

In [8]:
modes = 16
width = 64
mod_FNO = FNO1d(modes, width).to(device)

seq = 32

shape = (20,seq,1)
sample_FNO = torch.rand(*shape,device=device)

print('Yohoo, FNO created')

Yohoo, FNO created


In [9]:
import warnings ; warnings.filterwarnings("ignore")
rkgb_on_FNO = rkgb.make_all_graphs(mod_FNO,sample_FNO,check_device_is_gpu=False)

In [10]:
sg = rkgb_on_FNO.S_graph
pg = rkgb.Ptools.S_to_P_init(sg)
rkgb.Ptools.rule_group_sequences(pg)
rkgb.Ptools.rule_merge_small_flows(pg)

In [11]:
rkgb.print_graph(pg,name="P_graph FNO")

Partitioned forward graph : 19 nodes


kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
kf.service.services: KApplicationTrader: mimeType "x-scheme-handler/file" not found
